In [ ]:
!pip install numpy; pandas; scikit-learn


In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
project_dir = r"customer-churn-prediction\Research"
data_dir = "data" 

In [29]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(project_dir, data_dir, file_name)

    return pd.read_csv(file_path)

In [34]:
data = pd.read_csv(r"E:\1. Projects\ML-internship\customer-churn-prediction\Research\data\churn_modelling.csv")

In [35]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10002 non-null  int64  
 1   CustomerId       10002 non-null  int64  
 2   Surname          10002 non-null  object 
 3   CreditScore      10002 non-null  int64  
 4   Geography        10001 non-null  object 
 5   Gender           10002 non-null  object 
 6   Age              10001 non-null  float64
 7   Tenure           10002 non-null  int64  
 8   Balance          10002 non-null  float64
 9   NumOfProducts    10002 non-null  int64  
 10  HasCrCard        10001 non-null  float64
 11  IsActiveMember   10001 non-null  float64
 12  EstimatedSalary  10002 non-null  float64
 13  Exited           10002 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 1.1+ MB


In [37]:
data.shape

(10002, 14)

In [38]:
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                float64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard          float64
IsActiveMember     float64
EstimatedSalary    float64
Exited               int64
dtype: object

In [39]:
data.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64

In [54]:
columns = ['Geography', 'Age', 'HasCrCard', 'IsActiveMember']

nan_index_list = []
for col in columns:
    nan_index_list.append(data[data[col].isna()].index.to_list())
    
    
print(nan_index_list)



# data[data[].isna()].index.to_list()

[[6], [9], [4], [8]]


In [45]:
data.iloc[6, : ]

RowNumber                 7
CustomerId         15592531
Surname            Bartlett
CreditScore             822
Geography               NaN
Gender                 Male
Age                    50.0
Tenure                    7
Balance                 0.0
NumOfProducts             2
HasCrCard               1.0
IsActiveMember          1.0
EstimatedSalary     10062.8
Exited                    0
Name: 6, dtype: object

In [48]:
data.loc[data.duplicated(keep=False)].sort_values(['CustomerId'])

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
10001,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0
10000,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0
9999,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1


In [55]:
# Some data cleaning process

def clean_data(df):
    return(
        df
        .dropna()
        .drop_duplicates()
        .rename(columns = str.lower)
    )


In [56]:
cleaned_data = clean_data(data)

In [58]:
cleaned_data.shape

(9996, 14)

In [62]:
data.shape[0]-cleaned_data.shape[0]

6

In [65]:
cleaned_data.columns

Index(['rownumber', 'customerid', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
       'isactivemember', 'estimatedsalary', 'exited'],
      dtype='object')

In [88]:
num_col = [col for col in cleaned_data.columns if cleaned_data[col].dtypes != 'O']
cat_col = [col for col in cleaned_data.columns if cleaned_data[col].dtypes == 'O']

In [91]:
# cat_col

In [76]:
for col in cleaned_data.columns:
    print(f"Number of unique values in {col}: {cleaned_data[col].nunique()}")

Number of unique values in rownumber: 9996
Number of unique values in customerid: 9996
Number of unique values in surname: 2932
Number of unique values in creditscore: 460
Number of unique values in geography: 3
Number of unique values in gender: 2
Number of unique values in age: 73
Number of unique values in tenure: 11
Number of unique values in balance: 6379
Number of unique values in numofproducts: 4
Number of unique values in hascrcard: 2
Number of unique values in isactivemember: 2
Number of unique values in estimatedsalary: 9995
Number of unique values in exited: 2


In [90]:
# cleaned_data.columns